In [1]:
!pip install pytorchvideo
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 844.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 30.2 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=3f44311d4a5f9eb1492ec47aaa6007d95aff99e60bf5591e727b01c33779dd23
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=7df0fd57dcb83baa8bdb875f6d23850a14be5549914350afbbca163fb654a1c2
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
import torch
from typing import Optional, Sequence
from torch import Tensor
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_video

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize,ConvertImageDtype
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [3]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60







def transform_data(x):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                Normalize(([0.2933, 0.4799, 0.6723]), ([0.1576, 0.1674, 0.2477])),
                Grayscale(num_output_channels=1),
                CenterCrop([720,900]),
                Resize([512,512]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)
#                 Lambda(lambda x: print(x.shape))

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    global means,stds
    global nst
    
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)
 
    if video_data["video"] is None:
        return None        # or pass
    else:
    
        video_data["video"] = video_data["video"].permute(1,0,2,3)
        
        video_data = transform_data( video_data)

        return video_data["video"]

def load_dataloaders(batch_size,start,end):

    
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        time_start = []
        time_end = []
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


       
            
        
            
    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [4]:
def load_testloader(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y
    

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            

    signds = SignDataset(train_inputs, train_classes)
    
    

    testloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [5]:
def load_vid(batch_size,start,end):


    train_inputs =[] #x

    video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Val_Set/VALIDATION/MSSL_VAL_SET_VIDEOS/p01_n131.mp4")
    # file functions

    vid = lp_video(video,start, end)

    for m in torch.unbind(vid, dim=3):
        train_inputs.append(m)

    

    testloader = DataLoader(train_inputs, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""



__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=60, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(1, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

In [7]:

def evluate(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, data in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            print(pred)



In [8]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):
        
        ce_loss = torch.nn.functional.cross_entropy(input, target, reduction='none') # important to add reduction='none' to keep per-batch-item loss
        pt = torch.exp(-ce_loss)
        focal_loss = (self.weight*(1-pt)**self.gamma * ce_loss).mean()
        return focal_loss
        


In [9]:
def CB_loss(logits, labels):

    no_of_classes = 60
    beta = 0.9999
    samples_per_cls =torch.tensor([166.0, 370.0, 63.0, 232.0, 905.0, 102.0, 537.0, 993.0, 133.0, 154.0, 286.0, 435.0, 452.0, 779.0, 354.0, 136.0, 232.0, 2154.0, 374.0, 787.0, 337.0, 268.0, 954.0, 167.0, 543.0, 324.0, 669.0, 333.0, 211.0, 433.0, 140.0, 665.0, 107.0, 305.0, 162.0, 61.0, 1229.0, 958.0, 897.0, 543.0, 625.0, 334.0, 776.0, 111.0, 412.0, 583.0, 255.0, 103.0, 108.0, 201.0, 384.0, 632.0, 13.0, 322.0, 664.0, 435.0, 194.0, 322.0, 113.0, 462.0])
    
    logits.cpu()
    labels.cpu()
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes

    labels_one_hot = torch.nn.functional.one_hot(labels, no_of_classes).float().cpu()

    weights = torch.tensor(weights).float()
    weights = weights.unsqueeze(0)
    weights = weights.repeat(labels_one_hot.shape[0],1) * labels_one_hot
    weights = weights.sum(1)
    weights = weights.unsqueeze(1)
    weights = weights.repeat(1,no_of_classes)
    weights = torch.FloatTensor(weights.transpose(0,1)).cuda()
    
    return FocalLoss(weights.half())(logits,labels)
   

In [10]:
def train(model, device, train_loader,validloader,scheduler, optimizer,epochs,lr):
    print("Train start")
    breakout = False
    
    last_loss = 100
    patience = 3
    triggertimes = 0
    
    
    model.half()
    model.cuda()

    
    

    optimizer.param_groups[0]['lr'] = lr
    
    for epoch in range(epochs):
        model.train()
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = CB_loss(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        if optimizer.param_groups[0]['lr'] != 1e-3:    
            scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 93:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            current_loss = test(model,device,validloader, mode = "Validating")
            
        
            print("breakout")
            return model, optimizer
            
            

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        
        current_loss = test(model,device,validloader)
        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger Times for early stopping:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model, optimizer

        else:
            print('trigger times: 0')
            trigger_times = 0

        last_loss = current_loss
    
    
        
    return model, optimizer

In [11]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    test_loss = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            test_loss += CB_loss(output, target)
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)
    test_loss /= num_samples
    print(' Val Accuracy: {}/{} ({:.0f}%) Val Loss: {:.6f}'.format(correct, num_samples,100. * correct / num_samples, test_loss))
    return  test_loss

In [12]:
n_classes = 60


model = nn.Sequential(OrderedDict([
    ('frontend',effnetv2_m())
]))





# specify loss function (categorical cross-entropy)

# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



    
# weights =torch.tensor([166.0, 370.0, 63.0, 232.0, 905.0, 102.0, 537.0, 993.0, 133.0, 154.0, 286.0, 435.0, 452.0, 779.0, 354.0, 136.0, 232.0, 2154.0, 374.0, 787.0, 337.0, 268.0, 954.0, 167.0, 543.0, 324.0, 669.0, 333.0, 211.0, 433.0, 140.0, 665.0, 107.0, 305.0, 162.0, 61.0, 1229.0, 958.0, 897.0, 543.0, 625.0, 334.0, 776.0, 111.0, 412.0, 583.0, 255.0, 103.0, 108.0, 201.0, 384.0, 632.0, 13.0, 322.0, 664.0, 435.0, 194.0, 322.0, 113.0, 462.0,2330.0])

# weights = weights / weights.sum()
# #print(weights)
# weights = 1.0 / weights
# weights = weights / weights.sum()

gamma = 2.0



# criterion = nn.CrossEntropyLoss(weight = weights.half())
# criterion = FocalLoss(weights.half())

# state = torch.load("../input/sign-model/model_optimizer1.pt")
# model.load_state_dict(state['model_state_dict'])
# model.half()
# model.cuda()
# optimizer.load_state_dict(state['optimizer_state_dict'])

In [13]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [14]:
#summary(model, (1, 512, 640), device='cpu')#1334+986 =2330

In [15]:
trainloader, validloader = load_dataloaders(batch_size=16,start=0, end=10)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p01_n002
file p06_n019
file p06_n007
file p01_n053
file p04_n074
file p05_n077
file p05_n027
file p05_n022
file p01_n110
file p06_n036
Train start
Epoch: 1 , Training Accuracy: 123/2105 (6%) Training Loss: 0.087894
 Val Accuracy: 31/527 (6%) Val Loss: 0.093323
trigger times: 0
Epoch: 2 , Training Accuracy: 238/2105 (11%) Training Loss: 0.071988
 Val Accuracy: 68/527 (13%) Val Loss: 0.076843
trigger times: 0
Epoch: 3 , Training Accuracy: 398/2105 (19%) Training Loss: 0.060119
 Val Accuracy: 95/527 (18%) Val Loss: 0.057587
trigger times: 0
Epoch: 4 , Training Accuracy: 491/2105 (23%) Training Loss: 0.052325
 Val Accuracy: 126/527 (24%) Val Loss: 0.051788
trigger times: 0
Epoch: 5 , Training Accuracy: 609/2105 (29%) Training Loss: 0.047373
 Val Accuracy: 138/527 (26%) Val Loss: 0.048157
trigger times: 0
Epoch: 6 , Training Accuracy: 703/2105 (33%) Training Loss: 0.041782
 Val Accuracy: 160/527 (30%) Val Loss: 0.045624
trigger times: 0
Epoch: 7 , Training Accuracy: 867/2105 (41%) Trai

In [16]:
trainloader, validloader = load_dataloaders(batch_size=16,start=10, end=20)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p01_n069
file p06_n109
file p04_n089
file p01_n044
file p05_n037
file p04_n038
file p06_n055
file p01_n099
file p02_n066
file p04_n035
Train start
Epoch: 1 , Training Accuracy: 423/2094 (20%) Training Loss: 0.080125
 Val Accuracy: 136/524 (26%) Val Loss: 0.064758
trigger times: 0
Epoch: 2 , Training Accuracy: 841/2094 (40%) Training Loss: 0.039578
 Val Accuracy: 182/524 (35%) Val Loss: 0.050171
trigger times: 0
Epoch: 3 , Training Accuracy: 1213/2094 (58%) Training Loss: 0.021578
 Val Accuracy: 298/524 (57%) Val Loss: 0.021912
trigger times: 0
Epoch: 4 , Training Accuracy: 1385/2094 (66%) Training Loss: 0.016911
 Val Accuracy: 336/524 (64%) Val Loss: 0.018524
trigger times: 0
Epoch: 5 , Training Accuracy: 1469/2094 (70%) Training Loss: 0.014486
 Val Accuracy: 353/524 (67%) Val Loss: 0.016891
trigger times: 0
Epoch: 6 , Training Accuracy: 1568/2094 (75%) Training Loss: 0.012576
 Val Accuracy: 365/524 (70%) Val Loss: 0.015129
trigger times: 0
Epoch: 7 , Training Accuracy: 1615/2094 

In [17]:
trainloader, validloader = load_dataloaders(batch_size=16,start=20, end=30)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p01_n057
file p01_n105
file p04_n058
file p05_n043
file p06_n020
file p02_n001
file p04_n032
file p05_n084
file p01_n102
file p06_n096
Train start
Epoch: 1 , Training Accuracy: 376/1862 (20%) Training Loss: 0.078941
 Val Accuracy: 171/466 (37%) Val Loss: 0.043762
trigger times: 0
Epoch: 2 , Training Accuracy: 888/1862 (48%) Training Loss: 0.031008
 Val Accuracy: 113/466 (24%) Val Loss: 0.067627
Trigger Times for early stopping: 1
Epoch: 3 , Training Accuracy: 1257/1862 (68%) Training Loss: 0.015963
 Val Accuracy: 339/466 (73%) Val Loss: 0.013275
trigger times: 0
Epoch: 4 , Training Accuracy: 1390/1862 (75%) Training Loss: 0.012007
 Val Accuracy: 352/466 (76%) Val Loss: 0.011307
trigger times: 0
Epoch: 5 , Training Accuracy: 1440/1862 (77%) Training Loss: 0.010369
 Val Accuracy: 360/466 (77%) Val Loss: 0.010506
trigger times: 0
Epoch: 6 , Training Accuracy: 1491/1862 (80%) Training Loss: 0.009496
 Val Accuracy: 373/466 (80%) Val Loss: 0.010590
Trigger Times for early stopping: 1
Ep

In [18]:
trainloader, validloader = load_dataloaders(batch_size=16,start=30, end=40)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)


file p02_n104
file p01_n090
file p01_n040
file p04_n073
file p01_n067
file p05_n004
file p06_n061
file p05_n092
file p06_n015
file p04_n106
Train start
Epoch: 1 , Training Accuracy: 860/2112 (41%) Training Loss: 0.046394
 Val Accuracy: 277/529 (52%) Val Loss: 0.034515
trigger times: 0
Epoch: 2 , Training Accuracy: 1544/2112 (73%) Training Loss: 0.014609
 Val Accuracy: 294/529 (56%) Val Loss: 0.032745
trigger times: 0
Epoch: 3 , Training Accuracy: 1813/2112 (86%) Training Loss: 0.006836
 Val Accuracy: 464/529 (88%) Val Loss: 0.005558
trigger times: 0
Epoch: 4 , Training Accuracy: 1927/2112 (91%) Training Loss: 0.004489
 Val Accuracy: 476/529 (90%) Val Loss: 0.004681
trigger times: 0
Epoch: 5 , Training Accuracy: 1966/2112 (93%) Training Loss: 0.003396
 Val Accuracy: 479/529 (91%) Val Loss: 0.004322
breakout


In [19]:
trainloader, validloader = load_dataloaders(batch_size=16,start=40, end=50)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p05_n095
file p02_n072
file p05_n031
file p01_n060
file p05_n048
file p05_n029
file p01_n000
file p01_n050
file p05_n025
file p04_n011
Train start
Epoch: 1 , Training Accuracy: 917/2152 (43%) Training Loss: 0.047041
 Val Accuracy: 238/539 (44%) Val Loss: 0.040375
trigger times: 0
Epoch: 2 , Training Accuracy: 1593/2152 (74%) Training Loss: 0.012706
 Val Accuracy: 320/539 (59%) Val Loss: 0.021591
trigger times: 0
Epoch: 3 , Training Accuracy: 1852/2152 (86%) Training Loss: 0.006303
 Val Accuracy: 479/539 (89%) Val Loss: 0.004753
trigger times: 0
Epoch: 4 , Training Accuracy: 1993/2152 (93%) Training Loss: 0.003237
 Val Accuracy: 492/539 (91%) Val Loss: 0.003817
trigger times: 0
Epoch: 5 , Training Accuracy: 2011/2152 (93%) Training Loss: 0.002927
 Val Accuracy: 494/539 (92%) Val Loss: 0.003317
breakout


In [20]:
trainloader, validloader = load_dataloaders(batch_size=16,start=50, end=60)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p01_n023
file p01_n042
file p01_n111
file p05_n083
file p05_n094
file p04_n103
file p01_n093
file p04_n003
file p05_n078
file p01_n052
Train start
Epoch: 1 , Training Accuracy: 708/1156 (61%) Training Loss: 0.032812
 Val Accuracy: 163/289 (56%) Val Loss: 0.027420
trigger times: 0
Epoch: 2 , Training Accuracy: 1026/1156 (89%) Training Loss: 0.006326
 Val Accuracy: 231/289 (80%) Val Loss: 0.007420
trigger times: 0
Epoch: 3 , Training Accuracy: 1109/1156 (96%) Training Loss: 0.001676
 Val Accuracy: 275/289 (95%) Val Loss: 0.001676
breakout


In [21]:
trainloader, validloader = load_dataloaders(batch_size=16,start=60, end=70)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p02_n098
file p06_n013
file p05_n006
file p01_n063
file p02_n097
file p05_n065
file p01_n108
file p06_n018
file p05_n009
file p05_n071
Train start
Epoch: 1 , Training Accuracy: 785/1739 (45%) Training Loss: 0.048476
 Val Accuracy: 258/435 (59%) Val Loss: 0.025375
trigger times: 0
Epoch: 2 , Training Accuracy: 1386/1739 (80%) Training Loss: 0.011135
 Val Accuracy: 356/435 (82%) Val Loss: 0.010368
trigger times: 0
Epoch: 3 , Training Accuracy: 1594/1739 (92%) Training Loss: 0.003592
 Val Accuracy: 393/435 (90%) Val Loss: 0.004864
trigger times: 0
Epoch: 4 , Training Accuracy: 1625/1739 (93%) Training Loss: 0.002903
 Val Accuracy: 398/435 (91%) Val Loss: 0.004356
breakout


In [22]:
trainloader, validloader = load_dataloaders(batch_size=16,start=70, end=80)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p04_n091
file p05_n075
file p01_n012
file p05_n076
file p04_n005
file p04_n046
file p01_n045
file p05_n041
file p01_n008
file p02_n101
Train start
Epoch: 1 , Training Accuracy: 969/1809 (54%) Training Loss: 0.037591
 Val Accuracy: 234/453 (52%) Val Loss: 0.034637
trigger times: 0
Epoch: 2 , Training Accuracy: 1532/1809 (85%) Training Loss: 0.009030
 Val Accuracy: 383/453 (85%) Val Loss: 0.007378
trigger times: 0
Epoch: 3 , Training Accuracy: 1725/1809 (95%) Training Loss: 0.002369
 Val Accuracy: 417/453 (92%) Val Loss: 0.003265
breakout


In [23]:
trainloader, validloader = load_dataloaders(batch_size=16,start=28, end=30)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p01_n102
file p06_n096
Train start
Epoch: 1 , Training Accuracy: 125/332 (38%) Training Loss: 0.061469
 Val Accuracy: 12/84 (14%) Val Loss: 0.146240
trigger times: 0
Epoch: 2 , Training Accuracy: 264/332 (80%) Training Loss: 0.008834
 Val Accuracy: 44/84 (52%) Val Loss: 0.042236
trigger times: 0
Epoch: 3 , Training Accuracy: 293/332 (88%) Training Loss: 0.004346
 Val Accuracy: 72/84 (86%) Val Loss: 0.007935
trigger times: 0
Epoch: 4 , Training Accuracy: 308/332 (93%) Training Loss: 0.003076
 Val Accuracy: 74/84 (88%) Val Loss: 0.006844
trigger times: 0
Epoch: 5 , Training Accuracy: 302/332 (91%) Training Loss: 0.003467
 Val Accuracy: 72/84 (86%) Val Loss: 0.005493
trigger times: 0
Epoch: 6 , Training Accuracy: 310/332 (93%) Training Loss: 0.002723
 Val Accuracy: 73/84 (87%) Val Loss: 0.006680
breakout


In [24]:
trainloader, validloader = load_dataloaders(batch_size=16,start=80, end=90)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p01_n085
file p05_n064
file p05_n017
file p01_n028
file p02_n079
file p01_n010
file p01_n082
file p01_n107
file p02_n088
file p01_n087
Train start
Epoch: 1 , Training Accuracy: 497/1949 (26%) Training Loss: 0.080088
 Val Accuracy: 157/488 (32%) Val Loss: 0.049652
trigger times: 0
Epoch: 2 , Training Accuracy: 500/1949 (26%) Training Loss: 0.067394
 Val Accuracy: 92/488 (19%) Val Loss: 0.074402
Trigger Times for early stopping: 1
Epoch: 3 , Training Accuracy: 628/1949 (32%) Training Loss: 0.050374
 Val Accuracy: 174/488 (36%) Val Loss: 0.051788
trigger times: 0
Epoch: 4 , Training Accuracy: 711/1949 (36%) Training Loss: 0.043214
 Val Accuracy: 204/488 (42%) Val Loss: 0.045410
trigger times: 0
Epoch: 5 , Training Accuracy: 767/1949 (39%) Training Loss: 0.040177
 Val Accuracy: 205/488 (42%) Val Loss: 0.041870
trigger times: 0
Epoch: 6 , Training Accuracy: 860/1949 (44%) Training Loss: 0.036845
 Val Accuracy: 227/488 (47%) Val Loss: 0.038818
trigger times: 0
Epoch: 7 , Training Accura

In [25]:
trainloader, validloader = load_dataloaders(batch_size=16,start=90, end=100)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p05_n086
file p06_n024
file p05_n070
file p04_n016
file p04_n068
file p04_n026
file p06_n039
file p06_n014
file p05_n059
file p02_n051
Train start
Epoch: 1 , Training Accuracy: 697/1748 (40%) Training Loss: 0.050032
 Val Accuracy: 246/437 (56%) Val Loss: 0.028030
trigger times: 0
Epoch: 2 , Training Accuracy: 1252/1748 (72%) Training Loss: 0.014475
 Val Accuracy: 275/437 (63%) Val Loss: 0.020523
trigger times: 0
Epoch: 3 , Training Accuracy: 1457/1748 (83%) Training Loss: 0.007226
 Val Accuracy: 362/437 (83%) Val Loss: 0.009514
trigger times: 0
Epoch: 4 , Training Accuracy: 1520/1748 (87%) Training Loss: 0.005606
 Val Accuracy: 368/437 (84%) Val Loss: 0.007828
trigger times: 0
Epoch: 5 , Training Accuracy: 1550/1748 (89%) Training Loss: 0.004430
 Val Accuracy: 373/437 (85%) Val Loss: 0.007797
trigger times: 0
Epoch: 6 , Training Accuracy: 1562/1748 (89%) Training Loss: 0.004418
 Val Accuracy: 375/437 (86%) Val Loss: 0.007919
Trigger Times for early stopping: 1
Epoch: 7 , Training 

In [26]:
trainloader, validloader = load_dataloaders(batch_size=16,start=100, end=108)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p06_n056
file p04_n080
file p05_n062
file p04_n034
file p04_n049
file p06_n033
file p02_n100
file p04_n030
Train start
Epoch: 1 , Training Accuracy: 626/1263 (50%) Training Loss: 0.034528
 Val Accuracy: 232/316 (73%) Val Loss: 0.011246
trigger times: 0
Epoch: 2 , Training Accuracy: 1054/1263 (83%) Training Loss: 0.005957
 Val Accuracy: 284/316 (90%) Val Loss: 0.004120
trigger times: 0
Epoch: 3 , Training Accuracy: 1167/1263 (92%) Training Loss: 0.002587
 Val Accuracy: 293/316 (93%) Val Loss: 0.002394
trigger times: 0
Epoch: 4 , Training Accuracy: 1201/1263 (95%) Training Loss: 0.002158
 Val Accuracy: 297/316 (94%) Val Loss: 0.002142
breakout


In [27]:
trainloader, validloader = load_dataloaders(batch_size=16,start=26, end=30)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,validloader,scheduler,optimizer,30,1e-2)

file p04_n032
file p05_n084
file p01_n102
file p06_n096
Train start
Epoch: 1 , Training Accuracy: 374/881 (42%) Training Loss: 0.045083
 Val Accuracy: 136/221 (62%) Val Loss: 0.017426
trigger times: 0
Epoch: 2 , Training Accuracy: 690/881 (78%) Training Loss: 0.010378
 Val Accuracy: 183/221 (83%) Val Loss: 0.007427
trigger times: 0
Epoch: 3 , Training Accuracy: 803/881 (91%) Training Loss: 0.004221
 Val Accuracy: 201/221 (91%) Val Loss: 0.003416
trigger times: 0
Epoch: 4 , Training Accuracy: 811/881 (92%) Training Loss: 0.003453
 Val Accuracy: 204/221 (92%) Val Loss: 0.003267
trigger times: 0
Epoch: 5 , Training Accuracy: 821/881 (93%) Training Loss: 0.002941
 Val Accuracy: 200/221 (90%) Val Loss: 0.003618
breakout


In [28]:
trainloader = load_testloader(batch_size=16,start=108, end=112)#73 not included 
test(model,device,trainloader)

file p06_n054
file p05_n047
file p06_n021
file p01_n081
 Val Accuracy: 194/1013 (19%) Val Loss: 0.164185


tensor(0.1642, device='cuda:0', dtype=torch.float16)

In [29]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")

In [30]:
# trainloader= load_vid(batch_size=16,start=7000, end=12000)#73 not included 
# evluate(model,device,trainloader)